In [1]:
import json
import minsearch

In [2]:
with open('documents.json','rt') as f_in:
    docs_raw = json.load(f_in)

In [3]:
documents=[]

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)
    

In [4]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [5]:
index = minsearch.Index(
    text_fields=['question','text','section'],
                keyword_fields=['course']
)

In [6]:
q = 'the course has already started, can I still enroll?'

In [7]:
index.fit(documents)

In [8]:
boost = {'question': 3.0, 'section': 0.5}

results = index.search(
    query = q,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

In [9]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

### Now, add the LLM component

In [10]:
from openai import OpenAI

In [11]:
client = OpenAI()

In [ ]:
#testing that api key is in environment
import os
#os.environ

In [13]:
q

'the course has already started, can I still enroll?'

In [12]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role":"user","content":q}]
)

In [13]:
response

ChatCompletion(id='chatcmpl-BePCovfIvgP867Eum0clmBI046R1P', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Whether you can enroll in a course that has already started depends on several factors, such as the institution, the type of course, and the specific policies in place. Here are some general steps you can take:\n\n1. **Check the Policy**: Review the course's enrollment policies on the institution's website or in the course catalog, which might provide specific guidelines about late enrollment.\n\n2. **Contact the Instructor**: Reach out to the course instructor or professor to discuss your situation. They may be flexible and allow you to join if you demonstrate a strong commitment and readiness to catch up.\n\n3. **Speak with the Registrar or Admissions Office**: Contact the registrar or admissions office of the school to inquire about late enrollment procedures and any deadlines you need to be aware of.\n\n4. **Consider Catchi

In [14]:
# response.choices[0].message.content

### This is only using our knowledge base, need to connect to GPT

In [31]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION asked on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.  
If the CONTEXT doesn't contain the answer, output NONE.

QUESTION: {question}
CONTEXT: {context}
"""

In [32]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer:{doc['text']}\n\n"

In [33]:
print(context)

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer:Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer:Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related questions
question: Course - When will the course start?
answer:The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with t

In [34]:
prompt = prompt_template.format(question=q, context=context).strip()

In [35]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION asked on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.  
If the CONTEXT doesn't contain the answer, output NONE.

QUESTION: the course has already started, can I still enroll?
CONTEXT: section: General course-related questions
question: Course - Can I still join the course after the start date?
answer:Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer:Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your fi

### Now ready for GPT

In [36]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role":"user","content":prompt}]
)

In [37]:
response.choices[0].message.content

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

### the next phase is to simplify with functions

In [38]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query = query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )
    return results

In [39]:
def build_prompt(query,search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION asked on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.  
If the CONTEXT doesn't contain the answer, output NONE.

QUESTION: {question}
CONTEXT: {context}
""".strip()

    context = ""

    for doc in results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer:{doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [40]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role":"user","content":prompt}]
    )
    
    return response.choices[0].message.content    

In [41]:
query = 'the course already started, can I still enroll?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query,search_results)
    answer = llm(prompt)
    return answer

In [42]:
rag(query)

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."

In [47]:
rag('how do I run kafka?')

'NONE'

In [51]:
rag('do we need to install docker?')


'NONE'

### Now working with Elastic Search

In [22]:
from elasticsearch import Elasticsearch

In [23]:
es_client = Elasticsearch('http://localhost:9200')

In [24]:
es_client.info

<bound method Elasticsearch.info of <Elasticsearch(['http://localhost:9200'])>>

In [ ]:
es_client.info()

In [26]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

/tmp/ipykernel_8585/295183685.py:18: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es_client.indices.create(index=index_name, body=index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [27]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [28]:
from tqdm.auto import tqdm

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [29]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|███████████████████████████████████████████████████████████████████████████| 948/948 [00:04<00:00, 230.28it/s]


In [43]:
query

'the course already started, can I still enroll?'

In [53]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es_client.search(index=index_name, body=search_query)

    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [55]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query,search_results)
    answer = llm(prompt)
    return answer

In [56]:
rag(query)

/tmp/ipykernel_8585/4266433848.py:22: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(index=index_name, body=search_query)


"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute."